# Team 11: Predicting the Impact of Twitter Posts on Bitcoin Prices

**Group Members:**  Andrey Bartashevich (14586517), Finn Prins (13458434), Duco Trompert (14591227)

**Research Question:** Can Twitter (a.k.a. X) posts of famous people influence the prices of Bitcoin?

# Hypothesis: 
**Null hypothesis:** There is no impact of Twitter  posts by individuals with over one million followers on the Bitcoin price movements.

**Alternative Hypothesis:** There is an impact of Twitter posts by individuals with over one million followers on the Bitcoin price movements


**Potentialy change the 1 mill foloowers with 250 000 followers, because the dataset doesnt have enough values for users with 1 000 000 followers.**

# Data preprocessing

**Imports**

In [8]:
import pandas as pd
import numpy as np

In [9]:
tweet = pd.read_csv('data/Bitcoin_tweets.csv', lineterminator='\n')
tweet.head()

/var/folders/d6/qmvkgm9x2kq_2htc9f1yhtb00000gn/T/ipykernel_11708/1708110246.py:1: DtypeWarning: Columns (4,5,6,7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  tweet = pd.read_csv('data/Bitcoin_tweets.csv', lineterminator='\n')


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False
1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False
2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False
3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False
4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False


In [13]:
# tweet = pd.read_csv('data/Bitcoin_tweets_dataset_2.csv', engine='python')
tweet['date'] = pd.to_datetime(tweet['date'], errors='coerce')
tweet = tweet.dropna(subset=['date'])
# mill = tweet[tweet['user_followers'] > 250000]
# print(len(mill))
tweet[['date', 'user_name', 'user_followers', 'text']].head()



,date,user_name,user_followers,text
0,2021-02-10 23:59:04,DeSota Wilson,8534.0,Blue Ridge Bank shares halted by NYSE after #b...
1,2021-02-10 23:58:48,CryptoND,6769.0,"😎 Today, that's this #Thursday, we will do a ""..."
2,2021-02-10 23:54:48,Tdlmatias,128.0,"Guys evening, I have read this article about B..."
3,2021-02-10 23:54:33,Crypto is the future,625.0,$BTC A big chance in a billion! Price: \487264...
4,2021-02-10 23:54:06,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,1249.0,This network is secured by 9 508 nodes as of t...


In [14]:
bitcoin = pd.read_csv('data/btcusd_1-min_data.csv')
bitcoin.head()
bitcoin.rename(columns={'Timestamp' : 'date'}, inplace=True)
bitcoin['date'] = pd.to_datetime(bitcoin['date'], unit='s')
bitcoin[['date','Close','Volume']].head()
# print(bitcoin['Volume'])


,date,Close,Volume
0,2012-01-01 10:01:00,4.58,0.0
1,2012-01-01 10:02:00,4.58,0.0
2,2012-01-01 10:03:00,4.58,0.0
3,2012-01-01 10:04:00,4.58,0.0
4,2012-01-01 10:05:00,4.58,0.0


In [16]:
merged = pd.merge(tweet[['date', 'user_name', 'user_followers', 'text']],bitcoin[['date', 'Close', 'Volume']], on='date', how='inner')
# merged = merged[merged['user_followers'] > 250_000]
# print(len(merged))
# merged.head()
print(len(merged))


132493
